## OCR Simple

In [ ]:
from docling.document_converter import DocumentConverter

source = "TEST_OCR.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"

In [12]:
import datetime
import logging
import time
from pathlib import Path

In [13]:
import pandas as pd

In [14]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.utils.export import generate_multimodal_pages
from docling.utils.utils import create_hash

In [15]:
_log = logging.getLogger(__name__)

In [16]:
IMAGE_RESOLUTION_SCALE = 2.0

In [19]:
def main():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path("TEST_OCR.pdf")
    output_dir = Path("scratch")

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting AssembleOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)

    rows = []
    for (
        content_text,
        content_md,
        content_dt,
        page_cells,
        page_segments,
        page,
    ) in generate_multimodal_pages(conv_res):

        dpi = page._default_image_scale * 72

        rows.append(
            {
                "document": conv_res.input.file.name,
                "hash": conv_res.input.document_hash,
                "page_hash": create_hash(
                    conv_res.input.document_hash + ":" + str(page.page_no - 1)
                ),
                "image": {
                    "width": page.image.width,
                    "height": page.image.height,
                    "bytes": page.image.tobytes(),
                },
                "cells": page_cells,
                "contents": content_text,
                "contents_md": content_md,
                "contents_dt": content_dt,
                "segments": page_segments,
                "extra": {
                    "page_num": page.page_no + 1,
                    "width_in_points": page.size.width,
                    "height_in_points": page.size.height,
                    "dpi": dpi,
                },
            }
        )

    # Generate one parquet from all documents
    df = pd.json_normalize(rows)
    now = datetime.datetime.now()
    output_filename = output_dir / f"multimodal_{now:%Y-%m-%d_%H%M%S}.parquet"
    df.to_parquet(output_filename)

    end_time = time.time() - start_time

    _log.info(
        f"Document converted and multimodal pages generated in {end_time:.2f} seconds."
    )

    # This block demonstrates how the file can be opened with the HF datasets library
    # from datasets import Dataset
    # from PIL import Image
    # multimodal_df = pd.read_parquet(output_filename)

    # # Convert pandas DataFrame to Hugging Face Dataset and load bytes into image
    # dataset = Dataset.from_pandas(multimodal_df)
    # def transforms(examples):
    #     examples["image"] = Image.frombytes('RGB', (examples["image.width"], examples["image.height"]), examples["image.bytes"], 'raw')
    #     return examples
    # dataset = dataset.map(transforms)

In [20]:
if __name__ == "__main__":
    main()

INFO:docling.document_converter:Going to convert document batch...
Fetching 9 files: 100%|██████████| 9/9 [00:00<?, ?it/s]
INFO:docling.pipeline.base_pipeline:Processing document TEST_OCR.pdf
INFO:docling.document_converter:Finished converting document TEST_OCR.pdf in 93.55 sec.
INFO:__main__:Document converted and multimodal pages generated in 95.20 seconds.


In [21]:
from datasets import Dataset
from PIL import Image

multimodal_df = pd.read_parquet("scratch/multimodal_2024-12-04_171052.parquet")

INFO:datasets:PyTorch version 2.5.1 available.


In [22]:
# Convert pandas DataFrame to Hugging Face Dataset and load bytes into image
dataset = Dataset.from_pandas(multimodal_df)
def transforms(examples):
         examples["image"] = Image.frombytes('RGB', (examples["image.width"], examples["image.height"]), examples["image.bytes"], 'raw')
         return examples

dataset = dataset.map(transforms)

Map: 100%|██████████| 35/35 [00:03<00:00,  9.31 examples/s] 


In [26]:
dataset

Dataset({
    features: ['document', 'hash', 'page_hash', 'cells', 'contents', 'contents_md', 'contents_dt', 'segments', 'image.width', 'image.height', 'image.bytes', 'extra.page_num', 'extra.width_in_points', 'extra.height_in_points', 'extra.dpi', 'image'],
    num_rows: 35
})

In [61]:
for segment in dataset[0]["segments"]:
    if segment['label'] == 'picture':
        bbox = segment['bbox']
        print(segment)

{'bbox': [0.5034954095977584, 0.30377120201033775, 0.8892027412364686, 0.5058443088724156], 'data': [], 'index_in_doc': 13, 'label': 'picture', 'text': 'Figure 1. We aim to build a vision foundation model to enable extensive perception capabilities including spatial hierarchy and semantic granularity. To achieve this, a single unified model Florence-2 is pre-trained on our FLD-5B dataset encompassing a total of 5.4B comprehensive annotations across 126M images, which are collected by our Florence data engine.'}


In [67]:
image_width, image_height = dataset[0]['image'].size

# Convertir coordenadas normalizadas a píxeles
left = int(bbox[0] * image_width)
top = int(bbox[1] * image_height)
right = int(bbox[2] * image_width)
bottom = int(bbox[3] * image_height)

cropped_image = dataset[0]['image'].crop((left, top, right, bottom))

# Guardar o mostrar la región recortada
cropped_image.show()